In [ ]:
#Basic NMF - random initialisations

In [ ]:
#runs the library file common_init_ which contains the initialisations for common functions and libraries
%run common_init_

In [ ]:
#reading audio file for NMF
x, Fs = sf.read('FMP_C2_F10.wav')
# Audio(x, rate=Fs)

In [ ]:
#Taking default window size 4096 and hop size 512
N = 4096
H = 512
w = np.hanning(N)
X = librosa.core.stft(x,n_fft=N,hop_length=H, window=w)
print("Shape of X is :", X.shape)
Y = np.abs(X)**2
print("Shape of Y is :", Y.shape)

In [ ]:
#setting seed for random operations
np.random.seed(0)

In [ ]:
# For NMF, we break up V ~ W.H, so here, Y serves as the matrix V
t = time.time()
V = Y
# V = np.abs(X)
R = 8
eps = 1
W_, H_, W_ret = NMF(V,R,eps)
elapsed = time.time()-t
print("Time elapsed (sec) :", elapsed)

In [ ]:
# ###numba
# t = time.time()
# V = Y
# R = 8   
# eps = 1
# W_, H_ = nu_NMF(V,R,eps)
# elapsed = time.time()-t
# print("Time elapsed (sec) :", elapsed)

In [ ]:
T_coef = np.arange(X.shape[1]) * H / Fs
F_coef = np.arange(X.shape[0]) * Fs / N
duration = (len(x)//Fs) + 1
#left, right, lower and upper bounds for the plots
left = min(T_coef)
right = max(T_coef) + N / Fs
lower = min(F_coef)
upper = max(F_coef)
ratio_ = (upper/X.shape[0])

In [ ]:
#Plotting W and H matrices after random NMF
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
librosa.display.specshow(W_, y_axis='log', x_axis='frames', vmax=1, cmap='Blues', sr=Fs, hop_length=H)
plt.xticks(np.arange(0.5, R+0.5,1), np.arange(0,R,1))
plt.xlabel('components')
plt.title('W learnt randomly')


plt.subplot(1,2,2)
librosa.display.specshow(H_, x_axis='time',y_axis='frames', vmax=1e2, cmap='Blues', sr=Fs, hop_length=H)
plt.ylabel('components')
plt.yticks(np.arange(0.5, R+0.5,1), np.arange(0,R,1))
plt.title('H learnt randomly')

# plt.savefig('Cmaj_random_init_learnt.png', bbox_inches='tight')

In [ ]:
X_mat = np.abs(X)
u,s,vh = np.linalg.svd(X,full_matrices=True)
W = np.abs(u[:,0:8])
H = np.transpose(np.abs(vh[:,0:8]))

In [ ]:
## Same NMF function, just limiting the iteration counts, input is SVD matrix

def NMF_const12345(V, R, eps, W_prime, H_prime):
    (K, N) = V.shape
    print("Shape of V is (K x N) : ", K, N)
    W_temp = np.zeros((K, R), dtype=np.float64)
    H_temp = np.zeros((R,N), dtype=np.float64)
    W = np.reshape(W_prime, (K,R))
    np.random.seed(0)
    H = H_prime
    eps_test_H = np.inf
    eps_test_W = np.inf
    H_dist = []
    W_dist = []
    iter_ = 0
    run = True
    while(iter_<100):

#[1 0 0]
#[0 1 0]
#[0 0 1]

        temp_1 = np.dot(np.transpose(W),V)
        temp_2 = np.dot(np.dot(np.transpose(W),W), H) + np.finfo(float).eps
        H_temp = np.multiply(H, np.nan_to_num(np.divide(temp_1,temp_2)))
        #H_temp = np.nan_to_num(H_temp)
        temp_3 = np.dot(V,np.transpose(H_temp))
        temp_4 = np.dot(np.dot(W,H_temp),np.transpose(H_temp)) + np.finfo(float).eps
        W_temp = np.multiply(W, np.nan_to_num(np.divide(temp_3, temp_4)))
        W_temp = np.nan_to_num(W_temp)

#         eps_test_H = np.linalg.norm(H-H_temp)
#         eps_test_W = np.linalg.norm(W-W_temp)
#         H_dist = np.append(H_dist, eps_test_H)
#         W_dist = np.append(W_dist, eps_test_W)
        H = H_temp
        W = W_temp
        iter_ = iter_ + 1
#         if (eps_test_H < eps) and (eps_test_W < eps):
#             run = False
#         if(iter_ > 4000):
#             print('broken')
#             print(eps_test_W, eps_test_H)
#             break

#     print("Error in H is :", eps_test_W)
#     print("Error in W is :", eps_test_H)
    print("Number of interations :", iter_)
    return W, H



In [ ]:
#Plotting W and H matrices after random NMF
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
librosa.display.specshow(W, y_axis='log', x_axis='frames', vmax=0.25, cmap='Blues', sr=Fs, hop_length=H)
plt.xticks(np.arange(0.5, R+0.5,1), np.arange(0,R,1))
plt.xlabel('components')
plt.ylabel('Hz (log scale)')
plt.title('W initialised with SVD')


plt.subplot(1,2,2)
librosa.display.specshow(H, x_axis='time',y_axis='frames', vmax=0.15, cmap='Blues')
plt.ylabel('components')
plt.xlabel('Time (s)')
plt.yticks(np.arange(0.5, 8+0.5,1), np.arange(0,8,1))
plt.title('H initialised with SVD')

# plt.savefig('Cmaj_both_init_SVD_init.png', bbox_inches='tight')

In [ ]:
# For NMF, we break up V ~ W.H, so here, Y serves as the matrix V
t = time.time()
# V = X
V = X_mat
R = 8
eps = 1
#Here H is init by score informed method, W is still the harmonic informed, no  onsets yet
[W_score, H_score] = NMF_const12345(V,R,eps, W, H)
elapsed = time.time()-t
print("Time elapsed (sec) :", elapsed)

In [ ]:
recons_Y = np.dot(W_score, H_score)
np.linalg.norm(Y-recons_Y)

In [ ]:
################### second part ################
#This part has the randomly initialized NMF for example 'Gravity'

In [ ]:
x2, Fs2 = sf.read('gravity_intro.wav')
x2 = x2.astype(float)
x2 = x2.sum(axis=1) / 2

Audio(x2, rate=Fs2)

In [ ]:
#Repeat for intro wav file
#Here default window size is 8820 and hop length 2230
N = 8820
H = 2230
#H = 512
w = np.hanning(N)
X2 = librosa.core.stft(x2,n_fft=N,hop_length=H, window=w)
print("Shape of X is :", X2.shape)
Y2 = np.abs(X2)**2
print("Shape of Y is :", Y2.shape)

In [ ]:
t = time.time()
V2 = Y2
R2 = 15
eps = 1
W_2, H_2, W_ret2 = NMF(V2,R2,eps)
elapsed = time.time()-t
print("Time elapsed (sec) :", elapsed)

In [ ]:
# #Numba
# t = time.time()
# V2 = Y2
# R2 = 15
# eps = 1
# W_2, H_2 = nu_NMF(V2,R2,eps)
# elapsed = time.time()-t
# print("Time elapsed (sec) :", elapsed)

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
librosa.display.specshow(np.random.rand(X2.shape[1], R2), y_axis='linear', x_axis='frames', vmax=2, cmap='Blues', sr=Fs2, hop_length=H)
plt.xlabel('components')
plt.xticks(np.arange(0.5, R2+0.5,1), np.arange(0,R2,1))
plt.title('W randomly initialized')
plt.subplot(1,2,2)
librosa.display.specshow(np.random.rand(R2,X2.shape[1]), x_axis='time',y_axis='frames', vmax=2, cmap='Blues', sr=Fs2, hop_length=H)
plt.ylabel('components')
plt.yticks(np.arange(0.5, R2+0.5,1), np.arange(0,R2,1))
plt.title('H randomly initialized')

# plt.savefig('Gravity_random_init.png', bbox_inches='tight')


In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
librosa.display.specshow(W_2, y_axis='linear', x_axis='frames', vmax=0.01, cmap='Blues', sr=Fs2, hop_length=H)
plt.xlabel('components')
plt.xticks(np.arange(0.5, R2+0.5,1), np.arange(0,R2,1))
plt.title('W learnt randomly')
plt.subplot(1,2,2)
librosa.display.specshow(H_2, x_axis='time',y_axis='frames', vmax=1, cmap='Blues', sr=Fs2, hop_length=H)
plt.ylabel('components')
plt.yticks(np.arange(0.5, R2+0.5,1), np.arange(0,R2,1))
plt.title('H learnt randomly')


# plt.savefig('Gravity_random_init_learnt.png', bbox_inches='tight')

In [ ]:
phase_2 = np.angle(X2)
V_recons2 = np.dot(W_2,H_2)*np.exp(phase_2, dtype='complex')
intro_ = librosa.istft(X2, win_length=N, hop_length=H)
intro_recons = librosa.istft(V_recons2, win_length=N, hop_length=H)

In [ ]:
Audio(intro_recons, rate=Fs2)

In [ ]:
mask_total_num = np.dot(W_2, H_2)
mask_total_dem = mask_total_num + np.finfo(float).eps
mask_total = np.divide(mask_total_num, mask_total_dem)
new_Y2 = np.multiply(Y2, mask_total)

In [ ]:
phase_2 = np.angle(X2)
V_total_recons2 = new_Y2*np.exp(phase_2, dtype='complex')
total_aud_recons = librosa.istft(np.sqrt(V_total_recons2), win_length=N, hop_length=H)

In [ ]:
Audio(total_aud_recons, rate=Fs2)

In [ ]:
X_mat = np.abs(X2)
u,s,vh = np.linalg.svd(X_mat,full_matrices=True)
W = np.abs(u[:,0:15])
H = np.transpose(np.abs(vh[:,0:15]))

In [ ]:
# For NMF, we break up V ~ W.H, so here, Y serves as the matrix V
t = time.time()
#V = Y
V = X_mat
R = 15
eps = 1
#Here H is init by score informed method, W is still the harmonic informed, no  onsets yet
[W_score2, H_score2] = NMF_const_score(V,R,eps, W, H)
elapsed = time.time()-t
print("Time elapsed (sec) :", elapsed)

In [ ]:
# For NMF, we break up V ~ W.H, so here, Y serves as the matrix V
t = time.time()
#V = Y
V = X_mat
R = 15
eps = 1
#Here H is init by score informed method, W is still the harmonic informed, no  onsets yet
[W_score2, H_score2] = NMF_const12345(V,R,eps, W, H)
elapsed = time.time()-t
print("Time elapsed (sec) :", elapsed)

In [ ]:
recons_Y = np.dot(W_score2,H_score2)
np.linalg.norm(Y2-recons_Y)